# **1. Image Segmentation**
* 컴퓨터 비전 분야에서 이미지나 비디오의 디지털 데이터를 여러 개의 부분 또는 객체로 분할하는 기술
* 이미지의 중요한 요소들의 식별하고 각 요소를 개별적으로 분석할 수 있게 하는 것

# **2. Image Segmentation의 유형**
* Sementic Segmentation의
  * 이미지의 각 픽셀을 미리 정의된 클래스 레이블 중 하나로 분류
  * 예) 자율 주행자의 도로, 차선, 보행자 등을 식별
* Instance Segmentation
  * 동일한 클래스 내의 서로 다른 개체들을 개별적으로 식별
  * 예) 이미지 내의 있는 개별 문체의 수를 파악하고 각각 물체를 식별 및 추적하는 경우
* Panoptic Segmentation
  * Semantic Segmentation, Instance Segmentation을 결합한 형태
  * 배경과 같은 클래스를 처리하는 Sementic Segmentation과 개체를 구분하는 Instance Segmentation을 모두 수행
  * 예) 풍경 이미지에서 하늘, 도로, 나무와 사람, 자동차를 동시에 식별

https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=581

# **3. 차량 파손 검사 예제**

In [12]:
import os
import random
import shutil
import cv2
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
data_root ='/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation'
file_root = f'{data_root}/data'
cls_list = ['Scratched', 'Breakage', 'Separated', 'Crushed']
project_name = 'cd'

In [4]:
# data_root
# data_root/data
# data_root/cd/train -> images, labels
# data_root/cd/valid -> images, labels
# data_root/cd/test -> images, labels

In [16]:
train_root = f'{data_root}/{project_name}/train'
valid_root = f'{data_root}/{project_name}/valid'
test_root = f'{data_root}/{project_name}/test'

for folder in [train_root, valid_root, test_root]:
    if not os.path.exists(folder):
        os.makedirs(folder)
    for s in ['images', 'labels']:
        s_folder = f'{folder}/{s}'
        if not os.path.exists(s_folder):
            os.makedirs(s_folder)

In [6]:
# 라벨링 형태 변환(YOLO): [cls xc, yc, w, h]
# [[x1, y1], [x2, y2] ... ] -> [cls xc yc w h] : normalize
def json_to_yolo_polygon(polygon, w, h):
    yolo_list = []
    for p in polygon:
        # normalize
        yolo_list.append(p[0]/w)
        yolo_list.append(p[1]/h)
    return " ".join([str(x) for x in yolo_list])

In [7]:
# 데이터 불러오기
file_list = glob.glob(f'{file_root}/annotations/*.json')
random.seed(2024)
random.shuffle(file_list)
print(len(file_list))

1200


In [8]:
file_list[0]

'/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation/data/annotations/0413509_as-2547434.json'

In [9]:
# 라벨링 형태 변환 파일 생성(json -> txt)
mask_name = file_list[0].split('/')[-1].replace('json', 'txt')
result = []

with open(file_list[0], 'r') as json_file:
    data = json.load(json_file)
    h = data['images']['height']
    w = data['images']['width']
    for ann in data['annotations']:
        label = ann['damage']
        if label in cls_list:
            polygon_cood = ann['segmentation'][0][0][:-1]
            cood_string = json_to_yolo_polygon(polygon_cood, w, h)
            yolo_string = f'{cls_list.index(label)} {cood_string}'
            result.append(yolo_string)

In [10]:
result

['1 0.11375 0.575 0.14875 0.5816666666666667 0.17625 0.5916666666666667 0.1975 0.5883333333333334 0.23375 0.595 0.27 0.5983333333333334 0.3125 0.61 0.35875 0.5933333333333334 0.38125 0.5933333333333334 0.4075 0.6066666666666667 0.41125 0.6283333333333333 0.395 0.6283333333333333 0.36875 0.6183333333333333 0.34625 0.6333333333333333 0.31625 0.6366666666666667 0.27625 0.6233333333333333 0.2275 0.6133333333333333 0.17 0.61 0.1325 0.605',
 '0 0.32625 0.2866666666666667 0.3475 0.28 0.35625 0.32166666666666666 0.38125 0.31833333333333336 0.46 0.315 0.46625 0.35 0.43375 0.4716666666666667 0.42125 0.5216666666666666 0.4 0.545 0.40625 0.5933333333333334 0.38375 0.605 0.3725 0.5716666666666667 0.36125 0.5316666666666666 0.33625 0.43 0.315 0.31333333333333335']

In [11]:
# 라벨링 형태 변환 파일 생성(json -> txt)
if not os.path.isdir(f'{file_root}/labels'):
    os.mkdir(f'{file_root}/labels')

for file in tqdm(file_list):
    result = []
    with open(file, 'r') as json_file:
        data = json.load(json_file)
        h = data['images']['height']
        w = data['images']['width']
        for ann in data['annotations']:
            label = ann['damage']
            if label in cls_list:
                polygon_cood = ann['segmentation'][0][0][:-1]
                cood_string = json_to_yolo_polygon(polygon_cood, w, h)
                yolo_string = f'{cls_list.index(label)} {cood_string}'
                result.append(yolo_string)
    if result:
        save_path = file.replace('annotations', 'labels').replace('json', 'txt')
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(result))

100%|██████████| 1200/1200 [07:12<00:00,  2.78it/s]


In [12]:
file_list = glob.glob(f'{file_root}/labels/*.txt')
random.shuffle(file_list)
test_ratio = 0.1
num_file = len(file_list)

test_list = file_list[:int(num_file*test_ratio)]
valid_list = file_list[int(num_file*test_ratio):int(num_file*test_ratio)*2]
train_list = file_list[int(num_file*test_ratio)*2:]

for i in test_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{test_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'{file_root}/images/{img_name}'
    shutil.copyfile(img_path, f'{test_root}/images/{img_name}')

for i in valid_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{valid_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'{file_root}/images/{img_name}'
    shutil.copyfile(img_path, f'{valid_root}/images/{img_name}')

for i in train_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{train_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'{file_root}/images/{img_name}'
    shutil.copyfile(img_path, f'{train_root}/images/{img_name}')

In [13]:
project_root = '/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation'

In [14]:
%cd /content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation

/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation


In [5]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [36]:
import yaml
import ultralytics
from ultralytics import YOLO

In [17]:
ultralytics.checks()

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 34.8/78.2 GB disk)


In [18]:
data = dict()
data['train'] = train_root
data['val'] = valid_root
data['test'] = test_root
data['nc'] = len(cls_list)
data['names'] = cls_list

with open(f'{project_root}/car_damage.yaml', 'w') as f:
    yaml.dump(data, f)

In [19]:
%cd /content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation

/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation


In [20]:
model = YOLO('yolov8s-seg.yaml')
results = model.train(data='car_damage.yaml', epochs=100, batch=16, device=0, patience=30, name='yolo_s')

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.yaml, data=car_damage.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolo_s, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

100%|██████████| 755k/755k [00:00<00:00, 18.0MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 93.7MB/s]


AMP: checks passed ✅


...

    100/100      5.09G      1.714      3.068      2.053       1.79          4        640: 100%|██████████| 73/73 [00:51<00:00,  1.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05<00:00,  1.52it/s]

                   all        232        766      0.551      0.337      0.377      0.237      0.531      0.299      0.313      0.139



100 epochs completed in 1.722 hours.
Optimizer stripped from runs/segment/yolo_s/weights/last.pt, 23.9MB
Optimizer stripped from runs/segment/yolo_s/weights/best.pt, 23.9MB

Validating runs/segment/yolo_s/weights/best.pt...
Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,781,148 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:12<00:00,  1.62s/it]


                   all        232        766      0.549      0.337      0.377      0.237      0.532      0.299      0.314      0.139
             Scratched        198        454      0.493      0.322      0.348      0.213      0.478      0.275      0.283      0.107
              Breakage         81        116      0.608      0.379      0.442      0.289      0.585      0.345       0.38      0.184
             Separated         78        125      0.505      0.367      0.372      0.224      0.433      0.296      0.284      0.126
               Crushed         57         71      0.592      0.282      0.347      0.222      0.632      0.282      0.309      0.141
Speed: 0.5ms preprocess, 7.4ms inference, 0.0ms loss, 7.6ms postprocess per image
Results saved to runs/segment/yolo_s


In [7]:
%cd /content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation

/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation


In [9]:
project_root = '/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation'
result_floder = f'{project_root}/runs/segment'

In [10]:
model = YOLO(f'{result_floder}/yolo_s/weights/best.pt')

In [38]:
metrics = model.val(split='test')

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation/cd/test/labels.cache... 229 images, 0 backgrounds, 0 corrupt: 100%|██████████| 229/229 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:15<00:00,  1.01s/it]


                   all        229        765      0.154      0.156      0.086     0.0366      0.118      0.113     0.0522     0.0163
             Scratched        196        484      0.225      0.205      0.142     0.0534      0.206      0.169     0.0942     0.0257
              Breakage         77         99      0.131      0.111     0.0545     0.0242      0.112     0.0909     0.0392     0.0156
             Separated         71        107      0.176      0.243      0.126     0.0618      0.135      0.178     0.0729     0.0234
               Crushed         61         75     0.0838     0.0667     0.0218    0.00695     0.0177     0.0133    0.00252   0.000781
Speed: 2.8ms preprocess, 10.8ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to runs/segment/val5


In [14]:
%cd /content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation

/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation


In [17]:
test_file_list = glob.glob(f'{test_root}/images/*')
random.shuffle(test_file_list)

In [18]:
test_img = cv2.imread(test_file_list[0])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
result = model(img_src)[0]


0: 480x640 1 Scratched, 89.0ms
Speed: 2.6ms preprocess, 89.0ms inference, 16.3ms postprocess per image at shape (1, 3, 480, 640)


In [22]:
from skimage.draw import polygon2mask

In [29]:
test_img = cv2.imread(test_file_list[1])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
result = model(img_src)[0]

result_mask = np.zeros(test_img.shape[:2])
masks = result.masks
masks

for m in masks:
    polygon_coord = m.xy[0]
    # 주어진 이미지 크기와 폴리곤 좌표를 사용하여 해당 영역을 1로 채운 마스크를 생성
    # 나머지는 0으로 유지
    mask = polygon2mask(test_img.shape[:2], polygon_coord)
    # maximum(): 두 배열의 요소별 최대값을 반환하므로 여러 객체의 마스크가 겹치더라도 최대값을 유지
    result_mask = np.maximum(mask, result_mask)
    # 2D 마스크를 3D 배열로 변환하고 repeat() 사용해 동일한 값을 3개의 채널에 복사
result_mask = np.repeat(result_mask[:, :, np.newaxis], 3, -1)

plt.subplot(1, 2, 1)
plt.imshow(img_src)
plt.subplot(1, 2, 2)
plt.imshow(result_mask)
plt.show()


0: 480x640 1 Scratched, 17.1ms
Speed: 2.4ms preprocess, 17.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


In [34]:
test_img = cv2.imread(test_file_list[3])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
result = model(img_src)[0]

result_mask = np.zeros(test_img.shape[:2])
masks = result.masks
masks

for m in masks:
    polygon_coord = m.xy[0]
    # 주어진 이미지 크기와 폴리곤 좌표를 사용하여 해당 영역을 1로 채운 마스크를 생성
    # 나머지는 0으로 유지
    mask = polygon2mask(test_img.shape[:2], polygon_coord)
    # maximum(): 두 배열의 요소별 최대값을 반환하므로 여러 객체의 마스크가 겹치더라도 최대값을 유지
    result_mask = np.maximum(mask, result_mask)
    # 2D 마스크를 3D 배열로 변환하고 repeat() 사용해 동일한 값을 3개의 채널에 복사
result_mask = np.repeat(result_mask[:, :, np.newaxis], 3, -1)

plt.subplot(1, 2, 1)
plt.imshow(img_src)
plt.subplot(1, 2, 2)
plt.imshow(result_mask)
plt.show()


0: 640x480 2 Scratcheds, 1 Crushed, 16.2ms
Speed: 3.0ms preprocess, 16.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 480)


In [42]:
project_root = '/content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation'
result_floder = f'{project_root}/runs/segment'

model = YOLO(f'{result_floder}/yolo_s/weights/cd_pretrrained.pt')
metrics = model.val(split='test')

test_img = cv2.imread(test_file_list[3])
img_src = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
result = model(img_src)[0]

result_mask = np.zeros(test_img.shape[:2])
masks = result.masks
masks

for m in masks:
    polygon_coord = m.xy[0]
    # 주어진 이미지 크기와 폴리곤 좌표를 사용하여 해당 영역을 1로 채운 마스크를 생성
    # 나머지는 0으로 유지
    mask = polygon2mask(test_img.shape[:2], polygon_coord)
    # maximum(): 두 배열의 요소별 최대값을 반환하므로 여러 객체의 마스크가 겹치더라도 최대값을 유지
    result_mask = np.maximum(mask, result_mask)
    # 2D 마스크를 3D 배열로 변환하고 repeat() 사용해 동일한 값을 3개의 채널에 복사
result_mask = np.repeat(result_mask[:, :, np.newaxis], 3, -1)

plt.subplot(1, 2, 1)
plt.imshow(img_src)
plt.subplot(1, 2, 2)
plt.imshow(result_mask)
plt.show()

Ultralytics YOLOv8.2.77 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,781,148 parameters, 0 gradients, 42.4 GFLOPs


val: Scanning /content/drive/MyDrive/KDT/8. 컴퓨터 비전/9. Segmentation/cd/test/labels.cache... 229 images, 0 backgrounds, 0 corrupt: 100%|██████████| 229/229 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:13<00:00,  1.13it/s]


                   all        229        765      0.154      0.156      0.086     0.0366      0.118      0.113     0.0522     0.0163
             Scratched        196        484      0.225      0.205      0.142     0.0534      0.206      0.169     0.0942     0.0257
              Breakage         77         99      0.131      0.111     0.0545     0.0242      0.112     0.0909     0.0392     0.0156
             Separated         71        107      0.176      0.243      0.126     0.0618      0.135      0.178     0.0729     0.0234
               Crushed         61         75     0.0838     0.0667     0.0218    0.00695     0.0177     0.0133    0.00252   0.000781
Speed: 5.2ms preprocess, 10.1ms inference, 0.0ms loss, 4.9ms postprocess per image
Results saved to runs/segment/val9

0: 640x480 2 Scratcheds, 1 Crushed, 16.2ms
Speed: 2.9ms preprocess, 16.2ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 480)
